In [1]:
import numpy as np
import cv2
import tensorflow as tf
import pandas as pd

In [2]:
import os
from tqdm.notebook import tqdm

In [12]:
from skimage.metrics import structural_similarity as ssim
from sklearn.cluster import KMeans

In [3]:
from utils import helper_functions as hf
from utils import change_detection as cd
from utils import video_summarization_helper_functions as vshf
import Config as Config

In [4]:
test_directory = './videos/test videos/All Test Videos'

In [5]:
cols = ['file', 'type', 'f1', 'f2', 'f3', 'f4']
annotation_file = pd.read_csv('./test data/final_annotations.txt',sep=' ', header = None)
annotation_file = annotation_file.drop([1,3,5,7,9,11,12], axis=1)
annotation_file.columns=cols
annotation_file.head()

,file,type,f1,f2,f3,f4
0,Abuse028_x264.mp4,Abuse,165,240,-1,-1
1,Abuse030_x264.mp4,Abuse,1275,1360,-1,-1
2,Arrest001_x264.mp4,Arrest,1185,1485,-1,-1
3,Arrest007_x264.mp4,Arrest,1530,2160,-1,-1
4,Arrest024_x264.mp4,Arrest,1005,3105,-1,-1


In [6]:
files_in_directory = os.listdir(test_directory)

In [10]:
model = tf.keras.models.load_model(Config.FINE_TUNED_LSTM_AUTO_ENCODER_PATH)

2023-09-20 21:57:32.309669: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9798 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5


In [46]:
model = tf.keras.models.load_model('./auto_encoder5.hdf5')

In [43]:
global_model_array = []

In [47]:
global_model_array_AE_5 = []

In [48]:
for file in tqdm(files_in_directory):
    file_path = os.path.join(test_directory, file)
    file_name = os.path.basename(file_path) 
    
    
    if file_name.endswith('.mp4') and not file_name.startswith('Normal_Videos'):
        print(file_name)
        global_ssim_sum = 0
        total_sequence_length_for_ssim = 0
        
        
        row = annotation_file[annotation_file['file'] == file_name].iloc[0]
        f1_start, f1_end, f2_start, f2_end = row['f1'], row['f2'], row['f3'], row['f4']
        
        #Extracting all the frames
        video_frames_dict = hf.get_frames_from_video(file_path)
        
        frames_in_batches = hf.get_frames_in_batches(30, video_frames_dict)
        frame_sequence_tracker = hf.get_frame_sequence_tracker(frames_in_batches)
        significant_keys = cd.perform_change_detection(frame_sequence_tracker)
        
        cost_tracker = {}
        
        for key in significant_keys:
            significant_sequence = frame_sequence_tracker[key]
            reshaped_sequences = significant_sequence.reshape(3, 10, 256, 256, 1)
            for index, single_sequence in enumerate(reshaped_sequences):
                single_sequence = np.expand_dims(single_sequence, axis=0)
                reconstructed_sequence = model.predict(single_sequence)  
                ####SSIM#####
                # print(single_sequence[0][0].shape)
                # break
                ssim_sum = 0
                for i in range(10):
                    image1 = single_sequence[0, i, :, :, 0]
                    image2 = reconstructed_sequence[0, i, :, :, 0]
                    ssim_value, _ = ssim(image1, image2, full=True, channel_axis = -1)
                ssim_sum += ssim_value/10
                total_sequence_length_for_ssim += 1
                global_ssim_sum += ssim_sum
         
                # Calculating reconstruction cost for the single sequence
                cost = np.linalg.norm(single_sequence - reconstructed_sequence[0])
                cost_tracker[f'{key}_{index}'] = cost
                
        cost_list = hf.get_values_from_dictionary(cost_tracker)
        keys_to_consider = set()
        k_means_cost_list = np.array(cost_list).reshape(-1, 1)
        number_of_clusters = 2
        kmeans = KMeans(n_clusters=number_of_clusters)
        kmeans.fit(k_means_cost_list)
        labels = kmeans.labels_
        cluster_centers = kmeans.cluster_centers_
        anomalous_center = max(cluster_centers)
        anomalies = [cost for cost, label in zip(k_means_cost_list, labels) if cluster_centers[label] == anomalous_center]
        for anomaly_cost in anomalies:
            for key, value in cost_tracker.items():
                if value == anomaly_cost:
                    keys_to_consider.add(key)
                    
        cost_threshold = hf.calculate_prediction_cost_threshold(cost_list)
        original_frames_for_summarization = []

        frame_key_list = vshf.get_frame_key_list(keys_to_consider)
        
        final_frames_key_list = vshf.get_final_frames_key_list(frame_key_list)
        final_original_frame_number_list = []
        
        for key in final_frames_key_list:
            start, end = hf.get_original_frame_numbers(key,30)
            for j in range(start, end + 1):
                final_original_frame_number_list.append(j)
        
        final_original_frame_number_list = set(final_original_frame_number_list)
        final_original_frame_number_list = sorted(list(final_original_frame_number_list))
        
        positive_instances = 0
        inclusion_percentile = 0
        for frame in final_original_frame_number_list:
            if (frame >= f1_start and frame <= f1_end) or (frame >= f2_start and frame <= f2_end):
                positive_instances += 1
        
        inclusion_percentile = positive_instances/ len(final_original_frame_number_list)
        initial_video_length = len(video_frames_dict)/30
        final_video_length = len(final_original_frame_number_list)/30
        compactness = initial_video_length - final_video_length
        ssim_global = global_ssim_sum/total_sequence_length_for_ssim
        
        ssim_normalized = (ssim_global - 0)/1
        inclusion_percentile_normalized = (inclusion_percentile - 0)/(1)
        compactness_normalized = (compactness - 0)/(initial_video_length - 0)
        output_array = [ssim_global, inclusion_percentile, compactness]
        new_output_array = [ssim_normalized,inclusion_percentile_normalized, compactness_normalized]
        global_model_array_AE_5.append(new_output_array)
    
        


  0%|          | 0/22 [00:00<?, ?it/s]

Arrest001_x264.mp4
1/1 [==============================] - 0s 25ms/step
Fighting042_x264.mp4
1/1 [==============================] - 0s 26ms/step
Assault010_x264.mp4
1/1 [==============================] - 0s 26ms/step
Arson007_x264.mp4
1/1 [==============================] - 0s 25ms/step
Robbery102_x264.mp4
1/1 [==============================] - 0s 25ms/step
Explosion002_x264.mp4
1/1 [==============================] - 0s 26ms/step
Arson010_x264.mp4
1/1 [==============================] - 0s 26ms/step
Assault006_x264.mp4
1/1 [==============================] - 0s 25ms/step
Abuse028_x264.mp4
1/1 [==============================] - 0s 36ms/step
Burglary024_x264.mp4
1/1 [==============================] - 0s 26ms/step
Assault011_x264.mp4
1/1 [==============================] - 0s 31ms/step
Explosion004_x264.mp4
1/1 [==============================] - 0s 26ms/step
Fighting003_x264.mp4
1/1 [==============================] - 0s 25ms/step
Explosion029_x264.mp4
1/1 [==============================] - 0s 

In [45]:
global_model_array

[[0.08512210957705975, 0.09019607843137255, 0.785172704296546],
 [0.08146733237409044, 0.655952380952381, 0.6244970943227537],
 [0.0883653163115183, 0.15447154471544716, 0.9239661247450084],
 [0.08502858210177651, 1.0, 0.8416506717850288],
 [0.0871389874567588, 0.8333333333333334, 0.8029556650246306],
 [0.08696025647814312, 0.2222222222222222, 0.7981559930226764],
 [0.08408675491809846, 0.0, 0.829059829059829],
 [0.07799098157253198, 0.967032967032967, 0.6627964426877471],
 [0.07324124234063285, 0.25333333333333335, 0.7875354107648725],
 [0.08250432014465331, 0.8333333333333334, 0.9500693481276006],
 [0.07833131290318676, 0.05263157894736842, 0.7508741258741258],
 [0.08799841403961182, 0.5111111111111111, 0.9526813880126183],
 [0.07507578940618605, 0.687037037037037, 0.8259187620889749],
 [0.09260755827029546, 0.2, 0.9377593360995851],
 [0.07323095461048149, 0.0, 0.9861143253876418],
 [0.09129622181256612, 0.375, 0.8881640260950606],
 [0.08051898129781086, 0.21313131313131314, 0.871910

In [49]:
global_model_array_AE_5

[[0.0862698423365752, 0.09019607843137255, 0.785172704296546],
 [0.08007015894199238, 0.7064102564102565, 0.651318730442557],
 [0.09084953049818674, 0.15447154471544716, 0.9239661247450084],
 [0.08624625908476964, 1.0, 0.77447216890595],
 [0.08682955304781596, 0.8333333333333334, 0.8029556650246306],
 [0.08656191314969744, 0.2222222222222222, 0.7981559930226764],
 [0.08675473079536901, 0.0, 0.829059829059829],
 [0.07731016260562554, 0.97, 0.6294466403162056],
 [0.07495620704832531, 0.2111111111111111, 0.7450424929178471],
 [0.0824025332927704, 0.8333333333333334, 0.9500693481276006],
 [0.07915226064230269, 0.0, 0.8557692307692307],
 [0.08890761195444594, 0.5111111111111111, 0.9526813880126183],
 [0.0753378845396496, 0.47888888888888886, 0.7098646034816247],
 [0.0933137747976515, 0.16666666666666666, 0.9253112033195021],
 [0.07259043565502872, 0.0, 0.9861143253876418],
 [0.09115851481755576, 0.375, 0.8881640260950606],
 [0.08099028007189438, 0.23627450980392156, 0.8680294992883943]]

In [50]:
final_array = global_model_array.extend(global_model_array_AE_5)
final_array